In [1]:
import cv2
import imgaug as ia
import imgaug.augmenters as iaa


In [5]:
def show_image(title: str, img):
    cv2.imshow(title, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
ia.seed(1)
img = cv2.imread("../cycle_imgs/test/rec_3_0003.jpg")
show_image("sdf", img)


In [7]:
seq = iaa.Sequential(
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-25, 25),
        shear=(-8, 8)
    )
)

img_aug = seq(image=img)
img_aug
show_image("sdf", img_aug)

In [27]:
# rotate_aug = iaa.Rotate(-20)
rotate_aug = iaa.Rotate((-20, 20))
for i in range(10):
    img_rot = rotate_aug(image=img)
    cv2.imshow("rot", img_rot)
    cv2.waitKey(0)
# print(i)
cv2.destroyAllWindows()

In [173]:
def aug(img):
    rot_steps = [-18 ,-15 ,-12 ,-9 ,-6 ,-3, 3 ,6 ,9 ,12 ,15 ,18]
    x_translate_steps = [9, 18, 27, 36]
    
    images = []

    for angle in rot_steps:
        for x_trans in x_translate_steps:
            aug = iaa.Sequential([
                iaa.Rotate(angle),
                iaa.TranslateX(px=x_trans)
            ])
            aug_img = aug(image=img)
            images.append(aug_img)
    
    return images

In [174]:
aug_images = aug(img)

for x in aug_images:
    cv2.imshow("rot", x)
    if cv2.waitKey(0) == ord('q'):
        break
    
cv2.destroyAllWindows()

In [23]:
from typing import Tuple
import math

def xywhn2xyxy(x: float, y: float, w: float, h: float, img_w: int, img_h: int) -> Tuple[int]:
    """Convert normalised YOLO labels (x, y, width, height) to VOC format (xmin, ymin, xmax, ymax)"""
    x1 = round(img_w * (x - w / 2), 4)
    x2 = round(img_w * (x + w / 2), 4)
    y1 = round(img_h * (y - h / 2), 4)
    y2 = round(img_h * (y + h / 2), 4)
    return x1, y1, x2, y2

In [31]:
def xyxy2xywhn(x1: float, y1: float, x2: float, y2: float, img_w: int, img_h: int) -> Tuple[int]:
    """Convert VOC format (xmin, ymin, xmax, ymax) to normalised YOLO labels (x, y, width, height)"""
    x = round(((x1 + x2) / 2) / img_w, 6)
    y = round(((y1 + y2) / 2) / img_h, 6)
    w = round((x2 - x1) / img_w, 6)
    h = round((y2 - y1) / img_h, 6)
    return x, y , w, h


In [30]:
xyxy2xywhn(139.0833, 161.3510, 182.3161, 190.6874, 360, 640)
# desired output: 0.446388 0.27503 0.120091 0.045838

(0.446388, 0.27503, 0.120091, 0.045838)

In [32]:
xn, yn, wn, hn, = xyxy2xywhn(185.7784, 156.9244, 236.4376, 199.2780, 360, 640)
xywhn2xyxy(xn, yn, wn, hn, 360, 640)
# desired output: 185.7784, 156.9244, 236.4376, 199.2780

(185.7784, 156.9242, 236.4376, 199.2781)

In [68]:
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage


def get_bboxes_from_file(file, img_shape):
    h, w, _ = img_shape
    with open(file) as f:
        lines = list(map(lambda line: line.strip().split(), f.readlines()))
        bboxes = []
        
        for line in lines:
            label, cx_norm, cy_norm, w_norm, h_norm = map(float, line)
            label = str(int(label))
            # convert coords
            x1, y1, x2, y2 = xywhn2xyxy(x=cx_norm, y=cy_norm, w=w_norm, h=h_norm, img_w=w, img_h=h)
            # print(label, cx_norm, cy_norm, w_norm, h_norm)
            # print(x1, y1, x2, y2)
            bboxes.append(BoundingBox(x1=x1, y1=y1, x2=x2, y2=y2, label=label))
            # bboxes.append(BoundingBox(x1=x1, y1=y1, x2=x2, y2=y2))
            
        bbs = BoundingBoxesOnImage(bboxes, shape=(h, w))

    return bbs

In [25]:
img = cv2.imread("../cycle_imgs/test/rec_3_0003.jpg")
H, W, _ = img.shape

bbs = get_bboxes_from_file("/home/sibi/Downloads/annot/rec_3_0003.txt", img_h=H, img_w=W)

seq = iaa.Sequential([
    iaa.Multiply((1.2, 1.5)), # change brightness, doesn't affect BBs
    iaa.Affine(rotate=20)
])

img_aug, bbs_aug = seq(image=img, bounding_boxes=bbs)

for i in range(len(bbs.bounding_boxes)):
    before = bbs.bounding_boxes[i]
    after = bbs_aug.bounding_boxes[i]
    print("BB %d: (%.4f, %.4f, %.4f, %.4f) -> (%.4f, %.4f, %.4f, %.4f)" % (
        i,
        before.x1, before.y1, before.x2, before.y2,
        after.x1, after.y1, after.x2, after.y2)
    )

image_before = bbs.draw_on_image(img, size=2)
image_after = bbs_aug.draw_on_image(img_aug, size=2, color=[0, 0, 255])

show_image("before", image_before)
show_image("after", image_after)


1 0.446388 0.27503 0.120091 0.045838
139.0833 161.351 182.3161 190.6874
1 0.591591 0.124711 0.058147 0.024251
202.5063 72.0547 223.4392 87.5754
0 0.481895 0.350322 0.055399 0.027141
163.5104 215.521 183.454 232.8912
7 0.53372 0.482006 0.23768 0.230197
149.3568 234.8208 234.9216 382.1469
4 0.424273 0.216515 0.169935 0.165639
122.15 85.5651 183.3266 191.5741
BB 0: (139.0833, 161.3510, 182.3161, 190.6874) -> (185.7784, 156.9244, 236.4376, 199.2780)
BB 1: (202.5063, 72.0547, 223.4392, 87.5754) -> (280.6429, 94.7052, 305.6218, 116.4494)
BB 2: (163.5104, 215.5210, 183.4540, 232.8912) -> (194.2978, 216.1821, 218.9796, 239.3258)
BB 3: (149.3568, 234.8208, 234.9216, 382.1469) -> (129.9493, 229.4771, 260.7424, 397.1833)
BB 4: (122.1500, 85.5651, 183.3266, 191.5741) -> (169.5630, 79.9174, 263.3074, 200.4569)


In [16]:
bbaug1 = bbs_aug.bounding_boxes[0]
print(bbaug1)
bbaug1.x1, bbaug1.y1, bbaug1.x2, bbaug1.y2, bbaug1.label

BoundingBox(x1=185.9352, y1=156.5660, x2=236.2606, y2=198.5240, label=1)


(185.93523, 156.56604, 236.26059, 198.524, '1')

In [37]:
with open('out_bbox.txt', 'w') as f:
    for bbox in bbs_aug.bounding_boxes:
        x, y, w, h = xyxy2xywhn(x1=bbox.x1, y1=bbox.y1, x2=bbox.x2, y2=bbox.y2, img_w=360, img_h=640)
        label = bbox.label
        line = f'{label} {x} {y} {w} {h}\n'
        f.write(line)


In [69]:
def write_labels(label_path, bbs):
    with open(label_path, 'w') as f:
        for bbox in bbs.bounding_boxes:
            x, y, w, h = xyxy2xywhn(x1=bbox.x1, y1=bbox.y1, x2=bbox.x2, y2=bbox.y2, img_w=360, img_h=640)
            label = bbox.label
            line = f'{label} {x} {y} {w} {h}\n'
            f.write(line)


In [94]:
TRAIN_DIR = "test_ds/train"
VAL_DIR = "test_ds/val"

AUG_TRAIN_DIR = "test_ds_aug/train"
AUG_VAL_DIR = "test_ds_aug/val"

In [95]:
rot_steps = [-18 ,-15 ,-12 ,-9 ,-6 ,-3, 3 ,6 ,9 ,12 ,15 ,18]
x_translate_steps = [9, 18, 27, 36]

def aug(img_path, label_path):
    img = cv2.imread(img_path)
    bbs = get_bboxes_from_file(label_path, img_shape=img.shape)
    
    count = 0
    
    img_name, ext = img_path.split('/')[-1].split('.')

    aug_name = f'{img_name}_{count:02d}'
    aug_img_path = f'{AUG_TRAIN_DIR}/images/{aug_name}.{ext}'
    aug_label_path = f'{AUG_TRAIN_DIR}/labels/{aug_name}.txt'

    # save original image
    shutil.copy(img_path, f'{AUG_TRAIN_DIR}/images/')
    shutil.move(f'{AUG_TRAIN_DIR}/images/{img_name}.{ext}', f'{AUG_TRAIN_DIR}/images/{aug_name}.{ext}')
    # save original labels
    shutil.copy(label_path, f'{AUG_TRAIN_DIR}/labels/')
    shutil.move(f'{AUG_TRAIN_DIR}/labels/{img_name}.txt', f'{AUG_TRAIN_DIR}/labels/{aug_name}.txt')
    

    for angle in rot_steps:
        for x_trans in x_translate_steps:
            count += 1

            aug = iaa.Sequential([
                iaa.Rotate(angle),
                iaa.TranslateX(px=x_trans)
            ])
            
            # get augmented image and bboxes
            img_aug, bbs_aug = aug(image=img, bounding_boxes=bbs)
            
            aug_name = f'{img_name}_{count:02d}'
            aug_img_path = f'{AUG_TRAIN_DIR}/images/{aug_name}.{ext}'
            aug_label_path = f'{AUG_TRAIN_DIR}/labels/{aug_name}.txt'
            
            # save image
            cv2.imwrite(aug_img_path, img_aug)

            # save labels
            write_labels(aug_label_path, bbs_aug)


In [97]:
import os
import shutil
from tqdm import tqdm

for file in tqdm(os.scandir(f'{TRAIN_DIR}/images')):
    name, ext = file.name.split('.')
    label_path = f'{TRAIN_DIR}/labels/{name}.txt'
    
    aug(img_path=file.path, label_path=label_path)


# copy labels.txt file
shutil.copy(f"{TRAIN_DIR}/labels/labels.txt", f'{AUG_TRAIN_DIR}/labels')

88it [01:50,  1.26s/it]


'test_ds_aug/train/labels/labels.txt'